# Embeddings

В интернете есть много обученных моделей word2vec на разных копусах и для разных языков. Можно скачать и загрузить готовую модель. 
Например:
+ [Google news (английский)](https://code.google.com/archive/p/word2vec/)
+ [Репозиторий NLPL - много разных моделей на разных языках](http://vectors.nlpl.eu/repository/#)
+ [Википедия на разных языках](https://wikipedia2vec.github.io/wikipedia2vec/pretrained/)
+ [RusVectores (русский)](https://rusvectores.org/ru/models/)
+ и т.д.

Мы сегодня будем пользоваться самой маленькой моделью с RusVectores (**news_upos_cbow_300_2_2017** - 130 мб, около 200 тыс слов в словаре, обучалась на корпусе русскоязычных новостей).     
Хорошая практика - обучение на корпусе с аннотированными частями речи, помогает с проблемой омонимии. 

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
# умеет читать сразу из архива на сайте
# не нужно скачивать и распаковывать
model = KeyedVectors.load_word2vec_format('https://rusvectores.org/static/models/news_upos_cbow_300_2_2017.bin.gz', binary=True)

In [ ]:
# посмотреть словарь
model.vocab

In [ ]:
# посмотреть вектор слова
# model.wv['скоро_ADV']
model.get_vector('скоро_ADV')

## Операции с векторами

Мера косинусной близости (угол)          

+ Чем чаще слова встречаются в одинаковых контекстах, тем больше значение. 
![](https://erikbern.com/assets/2015/09/vector-model1.png)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
model.similarity('скоро_ADV', 'грустно_ADV')

In [ ]:
# контекстные синонимы
model.similarity('скоро_ADV', 'быстро_ADV')

In [ ]:
# антонимы
model.similarity('быстро_ADV', 'медленно_ADV')

Найти самые близкие слова

In [ ]:
# из всего словаря
model.most_similar('быстро_ADV')

In [ ]:
# самое близкое слово из списка
model.most_similar_to_given('быстро_ADV', ['красиво_ADV', 'быстрота_NOUN', 'быстрый_ADJ'])

Самые непохожие слова

In [ ]:
model.most_similar(negative=['красивый_ADJ'])

Какое слово лишнее

In [ ]:
model.doesnt_match(['кошка_NOUN', 'собака_NOUN', 'голубь_NOUN', 'трамвай_NOUN'])

Операции с векторами     

![](https://www.distilled.net/uploads/word2vec_chart.jpg)  

+ positive - все, что хотим прибавить 
+ negative - все, что хоти отнять

In [ ]:
# сын + женщина = ?
model.most_similar(positive=['сын_NOUN', 'женщина_NOUN'])

In [ ]:
# мать - женщина = ?
model.most_similar(positive=['мать_NOUN'], negative=['женщина_NOUN'])

In [ ]:
# кошка - женщина = ?
model.most_similar(positive=['кошка_NOUN'], negative=['женщина_NOUN'])

In [ ]:
# россия - москва + париж = ?
model.most_similar(positive=['россия_NOUN', 'париж_NOUN'], negative=['москва_NOUN'])

Обучить свою модель 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

In [ ]:
newsgroups = fetch_20newsgroups()

In [ ]:
df = pd.DataFrame()
df['text'] = newsgroups.data
df['target'] = newsgroups.target

In [ ]:
%%time
tokenized_documents = [word_tokenize(text.lower()) for text in df.text]

In [ ]:
from gensim.models import Word2Vec

Обучаем модель       
Подробно про параметры модели в [документации](https://radimrehurek.com/gensim/models/word2vec.html)

In [ ]:
%%time
my_model = Word2Vec(sentences=tokenized_documents, # список списков токенов для каждого документа
                    min_count=10, # игнорировать слова которые встречаются реже
                    sg=0, # какой алгоритм применить 0 - cbow, 1 - skipgram
                    workers=4, 
                    size=150) # длина вектора

Сохраняем модель:
+ Если не планируем дообучать, то можно сохранить только словарь векторов - объект `KeyedVectors`, хранится в атрибуте ***model.wv***
+ Если хотим дообучить, то сохраняем целиком (займет больше места)

In [ ]:
# только векторы
vectors = my_model.wv
vectors.save('my_vectors.bin')

In [ ]:
# всю модель
my_model.save('my_model.bin')

Подгружаем назад

In [ ]:
my_vectors = KeyedVectors.load('my_vectors.bin')

In [ ]:
my_vectors.get_vector('true')

In [ ]:
my_vectors.mhttps://i0.wp.com/yaronvazana.com/wp-content/uploads/2018/09/average-vectors.png?w=698ost_similar('nice')

+ Полученные векторы можно использовать в качестве признаков для обучения.   
+ Но в каждом документе разное количество слов. Необходимо привести все документы к одной размерности, чтобы получить матрицу объектов-признаков. 
+ Самый популярный способ - поэлементное усреднение векторов всех слов в документе. 
+ Итоговый результат - матрица ***(Кол-во объектов х Размерность вектора)***

![](https://i0.wp.com/yaronvazana.com/wp-content/uploads/2018/09/average-vectors.png?w=698)

Что еще делают:
+ Поэлементный минимум
+ Поэлементный максимум
+ Конкатенация всех векторов + подложка из 0 

In [ ]:
SEED = 123
np.random.seed(123)
df_train, df_test = train_test_split(df, train_size=0.5, test_size=0.3, stratify=df.target)

In [ ]:
my_vectors.get_vector('hhhhh')

In [ ]:
# получить вектор документа - усреднить векторы слов
def get_doc_vector(document):
    tokens = word_tokenize(document.lower())
    vectors = []
    for token in tokens:
        try:
            vector = my_vectors.get_vector(token)
            vectors.append(vector)
        except KeyError:
            continue
    return np.mean(vectors, axis=0)

In [ ]:
get_doc_vector(df_train.text[10]).shape

In [ ]:
%%time
X_train = np.matrix([get_doc_vector(document) for document in df_train.text])
X_test = np.matrix([get_doc_vector(document) for document in df_test.text])

In [ ]:
X_train.shape, X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier(n_jobs=-1, n_estimators=50)

In [ ]:
%%time
model.fit(X_train, df_train.target)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(df_test.target, y_pred, digits=3))